## Processing ETL des sets de données Olist

#### 1. Nettoyage: Opérations globales à tous les fichiers source

Dans cette partie je gère les nettoyages qui peuvent potentiellement s'appliquer à toutes les tables, sans faire de traitement spécifique pour telle ou telle table

In [9]:
# Import de dépendances et chargement des datasets en mémoire vive

import os
import pandas as pd
from utils import clean_data

data = {}

rt = './data'
csv_file_paths = os.listdir(rt)
for csv_path in csv_file_paths:
    df = pd.read_csv(f'{rt}/{csv_path}')
    df = clean_data(df) # Utilisation d'un heler générique de nettoyage
    source = csv_path.replace('olist_', '').replace('.csv', '').replace('_dataset', '')
    data[source] = df

/home/mathieu/prj/olist-learning/utils.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/utils.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/utils.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
 

#### 2. Nettoyage: Cas spécifiques

Dans cette partie du notebook j'applique des netoyages plus spécifiques, sur des tables en particulier:
- customers: suppression de doublons et d"une colonne non utilisée
- geolocation et customers: normalisation des noms de ville pour cohérence des éventuels PKI qui utiliseraient les noms de ville

In [10]:
# Nettoyage des doublons de données table customers
data['customers'] = data['customers'].drop_duplicates(subset=['customer_unique_id'], keep='first')
data['customers'] = data['customers'].drop(columns=['customer_unique_id'])

In [11]:
from utils import norm

# Normalise les noms de villes
data['geolocation'] = data['geolocation']['geolocation_city'].apply(norm)
data['customers'] = data['customers']['customer_city'].apply(norm)

#### 3. Transformations

La transformation principale que j'ai imaginée sur ce cas d'école a été d'ajouter les traductions de noms de catégories directement dans la table produits.

Le but étant de simplifier les requêtes devant filtrer ou grouper par catégorie, cela évite d'avoir à faire des jointures ultérieures pour récupérer les traductions.

Au départ j'ai même considéré faire cela pour cmplètement supprimer la table de traductions, mais j'ai décidé de la garder, dans le scénario où un utilisateur brésilien aurait besoin de faire des requêtes sur la base de données: dans ce cas les jointures resteront faisables.

In [12]:
# Transformation de la table des produits

# Merge des dataframes pour ajouter la traduction anglaise et éviter les jointures
data['products'] = data['products'].merge(
    data['product_category_name_translation'],
    how='left', # Type de jointure : left pour conserver tous les produits même sans traduction
    left_on='product_category_name', # Clé de jointure dans la table des produits
    right_on='product_category_name' # Clé de jointure dans la table de traduction
)

# Renommer la nouvelle colonne pour éviter les conflits
data['products'].rename(columns={'product_category_name_english': 'product_category_name_en'},
                inplace=True)

# Résultat final
print(data['products'][['product_category_name', 'product_category_name_en']].head())

   product_category_name product_category_name_en
0             perfumaria                perfumery
1                  artes                      art
2          esporte_lazer           sports_leisure
3                  bebes                     baby
4  utilidades_domesticas               housewares


#### 4. Chargement en base de données


Pour le chargement j'ai choisi d'utiliser sqlite3 pour gérer la partie création de tables en manuel avec des scripts SQL; ça permet d'avoir une création de tables qui inclue directement toutes les contraintes nécessaires, et c'est plus simple que d'utiliser la syntaxe de sqlalchemy, qui ajoute une couche d'abstraction supplémentaire à SQL.

In [13]:
import sqlite3

DB_PATH = "./olist.db"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Ouverture et exécution du script SQL de création de schéma de base de données
with open('./schema.sql', 'r') as f:
    schema_sql = f.read()
    cur.executescript(schema_sql)

In [ ]:
# Chargement des données nettoyées/transformées dans la base de données SQLite
for table_name, df in data.items():
    df.to_sql(table_name, con=conn, if_exists='replace', index=False)